In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py

%reload_ext autoreload
%autoreload 2

## Look at comp_dict directly

In [7]:
def make_df_from_popsyn(file_path = 'example_ffp.h5'):
    '''
    takes the compact object dictionary after PBHs are injected and returns
    a pandas DataFrame of all stars and their attributes
    '''
    #these keys don't contain stars so ignore them in the loop creating the df
    ignored_keys = {'add_pbh', 'lat_bin_edges', 'long_bin_edges'}
    with h5py.File(file_path, 'r') as f:
        
        #get the attributes of the stars
        phys_keys = np.array(list(f['l0b0'].dtype.names))
        #create the master dataframe
        df = pd.DataFrame(columns=phys_keys)

        for key in list(f.keys()):
            if key not in ignored_keys:
                num_stars = len(np.array(f[key]))
                #! 0 is a physical quantitiy so initialize with nans
                sub_arr = np.full((num_stars, len(phys_keys)), np.nan)
                # sub_arr = np.zeros((num_stars, len(phys_keys)))

                #loop through all the stars in this bin and add their attributes to an array
                for i, star in enumerate(np.array(f[key])):
                    sub_arr[i] = list(star) 
                #add the attributes of all the stars from this bin to the master dataframe
                df = pd.concat([df, pd.DataFrame(sub_arr, columns=phys_keys)])
    
    return df

def sample_ffp_mass(N_ffps: int) -> np.array:
    '''
    This is a dummy function currently. Will want to sample some distribution of masses for the ffps.
    For now, return an array of length N_ffps with fixed mass value in units of solar masses
    '''

    #Jupiter is about 1e-3 solar masses
    return np.ones(N_ffps) * 1e-3

def set_ffp_masses(df_ffps: pd.DataFrame) :
    '''
    Modifies the input dataframe to overwrite the masses of the ffps.
    '''
    N_objects = df_ffps.shape[0]
    masses = sample_ffp_mass(N_objects)
    df_ffps['mass'] = masses
    #PopSyCLE sets Zero age main sequence mass,zams_mass, equal to mass for PBHs. 
    #Following this convention for ffps
    df_ffps['zams_mass'] = masses


def set_ffp_photometry(df_ffps: pd.DataFrame):
    '''
    Modifies the input dataframe to overwrite the photometry of the ffps.
    '''
    N_objects = df_ffps.shape[0]
    photometry_ffps = np.zeros(N_objects)

    df_ffps['ubv_J'] = photometry_ffps
    df_ffps['ubv_H'] = photometry_ffps
    df_ffps['ubv_K'] = photometry_ffps
    df_ffps['ubv_U'] = photometry_ffps
    df_ffps['ubv_I'] = photometry_ffps
    df_ffps['ubv_B'] = photometry_ffps
    df_ffps['ubv_V'] = photometry_ffps
    df_ffps['ubv_R'] = photometry_ffps

def set_ffp_rem_id(df_ffps: pd.DataFrame):
    '''
    Modifies the input dataframe to overwrite the remnant id of the ffps.
    '''
    N_objects = df_ffps.shape[0]
    #pbhs are highest remnant id implemented in popsycle so far so set ffps to 105
    rem_ids = np.ones(N_objects) * 105
    df_ffps['rem_id'] = rem_ids

def set_ffp_pop_id(df_ffps: pd.DataFrame):
    '''
    Modifies the input dataframe to overwrite the pop id of the ffps.
    Set it to 10 since this is what is done for PBHs
    '''
    N_objects = df_ffps.shape[0]
    #Population ID (e.g. Disk, Halo, Bulge. See https://galaxia.sourceforge.net/Galaxia3pub.html for details) 'popid'.
    pop_ids = np.ones(N_objects) * 10

def set_ffp_misc(df_ffps: pd.DataFrame) -> pd.DataFrame:
    '''
    Modifies the input dataframe to overwrite some miscilanious attributes of the ffps
    that are not relevant for us. Set them to NaNs (this is what is done for PBHs I believe)
    '''

    # Bolometric magnitude: mbol, 
    # Surface gravity: grav,
    # Metalicity: feh,
    # log(age/yr): 'age',
    # Effective temperature: teff
    # Galactic Extinsion 'exbv'


    N_objects = df_ffps.shape[0]
    misc_ffps_nans = np.full(N_objects, np.NaN)
   
    df_ffps['mbol'] = misc_ffps_nans
    df_ffps['grav'] = misc_ffps_nans
    df_ffps['feh'] = misc_ffps_nans
    df_ffps['age'] = misc_ffps_nans
    df_ffps['teff'] = misc_ffps_nans
    df_ffps['exbv'] = misc_ffps_nans


def inject_ffp_params(df_ffps: pd.DataFrame) -> pd.DataFrame:
    '''
    Modifies the input dataframe to overwrite all relevant parameters for ffps
    '''
    set_ffp_masses(df_ffps)
    set_ffp_photometry(df_ffps)
    set_ffp_rem_id(df_ffps)
    set_ffp_misc(df_ffps)
    set_ffp_pop_id(df_ffps)

    #todo
    #Heliocentric velocities (in km/s): 'vx', 'vy', 'vz',
    #Radial velocity and proper motions: 'vr', 'mu_b', 'mu_lcosb'
    # Galactic positions 'rad', 'glat', 'glon'
    # Heliocentric positions: 'px', 'py', 'pz', 
    # positions related by:
    # comp_helio = synthetic.galactic_to_heliocentric(
    #             comp_dict["rad"], comp_dict["glat"], comp_dict["glon"]
    #         )
    #         comp_dict["px"], comp_dict["py"], comp_dict["pz"] = comp_helio
    # Object number within given bin: obj_id - Don't think we care about this but should revisit later - obj_id

def write_ffp_h5(df_combined: pd.DataFrame, file_path: str):
    '''
    #todo 
    Writes the input dataframe (combined df of pbh-injected pop_syn and our ffp version) to a h5 file in the appropriate format
    for PopSyCLE to calculate the events
    '''
    pass

In [3]:
df = make_df_from_popsyn('example_ffp.h5')

In [4]:
df.head()

,zams_mass,mass,px,py,pz,vx,vy,vz,age,popid,...,ubv_H,ubv_K,ubv_U,ubv_I,ubv_B,ubv_V,ubv_R,vr,mu_b,mu_lcosb
0,0.503571,0.503571,3.631762,0.062991,-0.065598,-20.581390,-24.072226,-12.524226,5.827383,0.0,...,5.938429,5.792072,11.798287,7.600787,10.721358,9.466644,8.552930,-20.766177,-0.748646,-1.375852
1,0.393893,0.393893,4.790543,0.082235,-0.086362,-18.358408,-10.715219,-3.930184,5.544013,0.0,...,6.613530,6.421209,12.853517,8.287896,11.697989,10.366171,9.388461,-18.465815,-0.187513,-0.457441
2,0.095672,0.095672,5.375013,0.092805,-0.097811,7.669304,-35.641441,-15.736011,5.257884,0.0,...,8.917903,8.649642,18.339668,11.017556,16.098106,14.173828,12.747360,7.337969,-0.611782,-1.402376
3,0.103781,0.103781,1.381481,0.024024,-0.025084,-18.424522,-6.588330,-14.462896,6.750011,0.0,...,8.849195,8.581558,17.990839,10.907817,15.848731,13.963803,12.583215,-18.270712,-2.257165,-0.956019
4,0.146732,0.146732,2.826867,0.049280,-0.051460,-30.147104,-21.643343,-0.324761,6.945432,0.0,...,8.702185,8.443503,16.352642,10.557643,14.792372,13.177802,12.001276,-30.508786,-0.065583,-1.574057


In [5]:
#create a copy of the master dataframe to modify with ffp parameters
df_ffps = df.copy(deep=True)

In [8]:
inject_ffp_params(df_ffps)

In [9]:
df_ffps.tail()

,zams_mass,mass,px,py,pz,vx,vy,vz,age,popid,...,ubv_H,ubv_K,ubv_U,ubv_I,ubv_B,ubv_V,ubv_R,vr,mu_b,mu_lcosb
189125,0.001,0.001,10.514203,0.185000,-0.186309,282.379327,240.255768,-140.221721,NaN,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,282.496429,-2.808689,4.813666
189126,0.001,0.001,11.476559,0.201334,-0.202588,153.757345,-160.571399,-233.106288,NaN,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,153.779971,-4.279472,-2.949307
189127,0.001,0.001,9.307258,0.162936,-0.164345,-33.157449,337.387157,198.751973,NaN,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-33.115607,4.499910,7.639347
189128,0.001,0.001,7.250479,0.127516,-0.129697,184.769714,107.161942,82.733854,NaN,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,184.776763,2.406319,3.112992
189129,0.001,0.001,8.154490,0.144015,-0.145642,94.004530,-247.763986,5.968452,NaN,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.926298,0.154997,-6.403624


In [19]:
#See how the original dataframe treats PBHs for comparison
df.loc[df['rem_id']==104][['mbol', 'grav', 'feh', 'age', 'teff', 'exbv']]

,mbol,grav,feh,age,teff,exbv
188417,NaN,NaN,NaN,NaN,NaN,NaN
188418,NaN,NaN,NaN,NaN,NaN,NaN
188419,NaN,NaN,NaN,NaN,NaN,NaN
188420,NaN,NaN,NaN,NaN,NaN,NaN
188421,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
189125,NaN,NaN,NaN,NaN,NaN,NaN
189126,NaN,NaN,NaN,NaN,NaN,NaN
189127,NaN,NaN,NaN,NaN,NaN,NaN
189128,NaN,NaN,NaN,NaN,NaN,NaN


Want to set t_eff (temp), all ubvs and ztfs (photometry) = 0, set velocities: vx,vy,vz. set positions: px,pv,pz, rad, glat, glon. Assign new population id popid, sample mass from distribution (zams_mass vs mass are always almost the same it seems)